# Update existing Sfincs model with boundary conditions and forcing

This notebook demonstrates how to update an existing SFINCS model with gauges, boundary conditions and forcing such as precipitation. The notebook does not contain the SFINCS code or executables to run the model with. 

We will perform the following:
* append an existing schematization that only contains static schematization layers
* introduce gridded precipitation only 
* introduce gridded precipitation, water level boundaries and observation gauge point locations
* write the new models to new model folders

This notebook demonstrates the update functionalities on a test model delivered with the hydromt_sfincs plugin.

First let us check if HydroMT has the Sfincs plugin available. The result of the code block below should be
`hydroMT model plugins: sfincs (vx.x.x)` where `x.x.x` denotes the current version (e.g. `0.1.0`). If Sfincs seems missing, ensure it is installed by typing `pip install hydromt_sfincs` on a command line.

In [ ]:
!hydromt --models

### Add precipitation forcing to an existing model
Rather than rebuilding a model from scratch, we can add or update components to an existing model. You may want to update several things at the same time. This may be useful, for instance when you are considering model intercomparisons, with different parameter sets or underlying static maps, or comparing the impact of different forcing datasets. 

This is possible by preparing a **configuration file** that includes every components and settings that you want to do during your update.

The ini-file contains the model setup configuration and determines which components are updated and in which sequence and sets optional arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`.

Each header as shown between `[...]` (e.g. `[setup_p_forcing_from_grid]`) corresponds with a model component which are explained in the [docs(model_components)](https://deltares.github.io/hydromt_sfincs/latest/user_guide/sfincs.html).

Let's open the example configuration file **sfincs_update_precip.ini** from the model repository [examples folder] and have a look at the settings.

In [ ]:
fn_ini = "sfincs_update_precip.ini"
with open(fn_ini, "r") as f:
    txt = f.read()
print(txt)

Two components are to be updated:
* `[setup_config]`: This section directly overwrites parts of the .inp configuration file. You can extend this with any settings of the .inp file. The time settings need to be altered. We have made sure that the time coverage is available in the precipitation sample dataset. 
* `[setup_p_forcing_from_grid]`: Here, we identify which dataset, available in our data catalog is to be used as forcing. The dataset **era5_hourly** has to be represented in the **data_catalog.yml**. Please check your **$USER_HOME/.hydromt_data** folder to see if it is indeed present. If it is, then you can continue below to add the forcing. 

We need the `update` command of hydromt to update the model with this .ini file. Let's get an overview of the options.

In [ ]:
!hydromt update --help

### Update sfincs precipitation
Now we will add the precipitation and write the model to a new folder

In [ ]:
!hydromt update sfincs ./sfincs_coastal -o ./sfincs_coastal_precip -i sfincs_update_precip.ini -v

The example above means the following: run **hydromt** with:

- `update sfincs`: i.e. update a sfincs model
- `./venice_sfincs`: original model folder
- `-o ./venice_sfincs_precip`: output updated model folder
- `-i sfincs_update_precip.ini`: setup configuration file containing the components to update and their different options
- `v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Let's inspect the new model folder

In [ ]:
!ls ./sfincs_coastal

We now have a **precip.nc** file with the precipitation. The .inp file now also contains a reference to this dataset.

### Inspect forcing data

The forcing data will also be available for analysis and plotting within hydromt. The `plot_forcing` method plots the area averaged precipitation. If you are familiar with xarray and matplotlib, it is straighforward to make some other plots as well.

In [ ]:
%matplotlib inline
from hydromt_sfincs import SfincsModel
import matplotlib.pyplot as plt

Below, we first plot the area averaged time series

In [ ]:
mod = SfincsModel(root="sfincs_coastal_precip", mode="r")
mod.forcing.pop("bzs", None)  # let's focus on precip for now
_ = mod.plot_forcing()

We can also manipulate the forcing and make spatial plots with [xarray](https://xarray.pydata.org/en/stable/) logic. Below we plot the time accumulated rainfall over the entire period.

In [ ]:
# precipitation forcing is in the key "netampr"
precip_sum = mod.forcing["netampr"].sum(dim="time")
precip_sum.attrs.update(unit="mm")
_ = precip_sum.plot()

We have another .ini file that also includes water level boundaries from the Global Tide and Surge Model, and also includes several measurement locations (`setup_gauges`). Below we print the contents of this .ini file and the gauges file.

In [ ]:
fn_ini = "sfincs_update_forcing.ini"
with open(fn_ini, "r") as f:
    txt = f.read()
print(txt)

In [ ]:
# content of gauges
import pandas as pd

fn_gauges = "data/venice_gauges.csv"
df = pd.read_csv(fn_gauges)
df

You can see a few things:
* the water level forcing configured under `setup_h_forcing` refers to a dataset from the catalog. Check the .yml of the catalog (see folder `$USER_HOME/.hydromt_data` for further details)
* the file `venice_gauges.csv` has a very simple setup with latitude/longitude coordinate pairs

Now let's build another version of the model that includes these new inputs

In [ ]:
!hydromt update sfincs ./sfincs_coastal -o ./sfincs_coastal_forcing -i sfincs_update_forcing.ini -v

We can now again inspect the model

In [ ]:
# define the model's configuration file
root = r"./sfincs_coastal_forcing"

# read the model with hydromt sfincs methods
mod = SfincsModel(root=root, mode="r")
mod.read()
_ = mod.plot_basemap(figsize=(13, 7))

We also updated the tidal boundary conditions from gtsm. Below we also inspect these:

In [ ]:
_ = mod.plot_forcing()